In [1]:
from dash import Dash, html, dcc, callback, Output, Input, ctx
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [2]:
from platform import python_version
import sys
print('Python: ' + python_version()) # Python: 3.12.
print ('dash: ' + sys.modules["dash"].__version__) # dash: 2.17.0
print('pandas: ' + pd.__version__) # pandas: 2.2.1
print('numpy: ' + np.__version__) # numpy: 1.26.4
print ('plotly: ' + sys.modules["plotly"].__version__) # plotly: 5.22.0

Python: 3.12.2
dash: 2.17.0
pandas: 2.2.1
numpy: 1.26.4
plotly: 5.22.0


## Import dataset and merge to dataframe

In [3]:
dataset_precip = pd.read_pickle("./data/UMAP_location.pkl")
datacube_precip = np.load("./data/WaterPrecip_datacube_large.npy")

In [4]:
datacube_precip = datacube_precip[datacube_precip.min(axis = (1,2)) >= 0,:,:]
datacube_precip = datacube_precip.reshape(-1,180*360).T
dataset_precip['Precip'] = list(datacube_precip)
del datacube_precip 

## Create a geojson grid of 1° Latitude x 1° Longitude
### geojson file format:
```python
# Feature collection object
{'type': 'FeatureCollection', 
 # List of features
 'features': [
     {'type': 'Feature',
      # supported geometry types: Point, LineString, Polygon, MultiPoint, MultiLineString, and MultiPolygon.
      'geometry':
          {'type': 'Polygon', 
            # List of coordinates
           'coordinates': 
               [[[lon_1, lat_1], [lon_2, lat_2], ....]]}
      # Dictionary of property key values sets
      'properties': {key: value},
      'id': x},
     .... # further features
 ]
}
```

In [5]:
# create an empty feature collection
geojson = {'type': 'FeatureCollection', 
                'features': []}
# create a feature for each 1° Latitude x 1° Longitude square
# 0° x 0° are at the coner of the grid
for x in range(180):
    for y in range(360):
        # Add the corners of the 1x1 square to a geometry object
        # The lat and lon are added as 90N -90S -180W 180E format
        temp_geometry = {'type': 'Polygon',
                 'coordinates': [[[y-180, x-90],
                                 [y-179, x-90],
                                 [y-179, x-89],
                                 [y-180, x-89]]]}
        # Add the geographical coordinates to the feature
        temp_features = {'type': 'Feature', 
                         'geometry': temp_geometry,
                         # Dictionary of property key values sets
                         # an easy readable string is choosen as feature name
                         'properties': {'location': "lat: " + str(x) + " lon: " + str(y)},
                         # unique int id for each feature
                         'id': x*360+y}
        # add feature to feature collection
        geojson["features"].append(temp_features)

In [6]:
# rename the column location to match the geojson feature name
dataset_precip["location"] = dataset_precip.apply(lambda row: "lat: " + str(row["lat"]) + " lon: " +  str(row["lon"]), axis=1)

## Create dashboard

In [7]:
global_figure_styles = dict(margin={"r":0,"t":0,"l":0,"b":0}, paper_bgcolor= '#e4edf4')

In [8]:
# Create a Dash application instance
app = Dash(__name__)
# Define the layout of the app
app.layout = html.Div([
    # Header for the application
    html.H1(children='Data Visualisation global Precipitation', style={'textAlign':'center'}),
    # Div for the precipitation map and its controls
    html.Div([
        html.H3(children='Map of precipitation', style={'margin-top': 0, 'margin-bottom': 0}),
        dcc.RadioItems(
            id='Time Aggregation', 
            options=["Mean", "Min", "Max", "Median"],
            value="Mean",
            inline=True
        ),
        dcc.Graph(id="graph_map")],
        style={'display': 'inline-block', 'width': '49%'}
    ),
    # Div for the UMAP visualization and its controls
    html.Div([
        html.H3(children='UMAP of precipitation', style={'margin-top': 0, 'margin-bottom': 0}),
        dcc.RadioItems(
            id='Dimension', 
            options=["2D", "3D"],
            value="2D",
            inline=True,
            style={'display': 'inline-block', 'width': 100}
        ),
        dcc.RadioItems(
            id='Location Aggregation', 
            options=["lat", "lon", "Mean", "Median"],
            value="lat",
            inline=True,
            style={'display': 'inline-block', 'border-left': '2px solid black'}
        ),
        dcc.Graph(id="graph_UMAP")],
        style={'display': 'inline-block', 'width': '49%'}
    ),
    html.Hr(),
    # Graph component to display the time series plot
    html.H3(children='Precipitation over time', style={'margin-top': 0, 'margin-bottom': 0}),
    dcc.Graph(id="graph_time"),
]),


# Callback function to create the UMAP plot based on user input
@callback(
    Output("graph_UMAP", "figure"), 
    Input('Dimension', 'value'),
    Input('Location Aggregation', 'value'),
)
def display_umap(dim, timeAgg):
    if dim == "3D":
        # Create a 3D scatter plot
        fig = px.scatter_3d(
            dataset_precip, x="UMAP_1", y="UMAP_2", z="UMAP_3",
            color= timeAgg, labels={'color': 'digit'},
            hover_name    = "location"
        )

    else:
        # Create a 2D scatter plot
        fig = px.scatter(
            dataset_precip, x="UMAP_1", y="UMAP_2",
            color= timeAgg, labels={'color': 'digit'},
            hover_name   = "location"
        )
    
    fig.update_layout(**global_figure_styles,
                      clickmode='event+select')
    return fig


# Callback function to create the choropleth map based on user input
@callback(
    Output("graph_map", "figure"),
    Input('Time Aggregation', 'value'),
)
def display_choropleth(timeAgg):
    temp_scale = [(0, "white"), (0.5, "blue"), (1, "navy")]
    fig = px.choropleth_mapbox(dataset_precip, geojson = geojson, color = timeAgg, opacity=.2,
                               color_continuous_scale = temp_scale,
                               locations = "location", featureidkey = "properties.location", 
                               center = {"lat": 50, "lon": 10}, 
                               mapbox_style = "carto-positron", zoom=3)
    fig.update_layout(**global_figure_styles,
                     clickmode='event+select')
    return fig

# Callback function to update the time series plot based on click data from other plots
@callback(
    Output("graph_time", "figure"),
    Input('Dimension', 'value'),
    Input("graph_map", "clickData"),
    Input("graph_UMAP", "clickData")
)
def display_line(dim, sel_map, sel_UMAP):
    # callback_context is used to determine the id of the component that triggered the callback.
    click = ctx.triggered_id if not None else 'graph_UMAP'
    itemp = 0
    # Check if the UMAP plot triggered the callback and a point is selected
    if click == "graph_UMAP" :
        if sel_UMAP is not None:
            itemp = sel_UMAP['points'][0]['pointNumber']
    # Check if the map plot triggered the callback and a point is selected
    elif click == "graph_map":
        if sel_map is not None:
            itemp = sel_map['points'][0]['pointNumber']
    
    fig = go.Figure(data=go.Scatter(y = list(dataset_precip.loc[itemp, "Precip"])))
    fig.update_layout(**global_figure_styles, 
                      height=300)
    return fig

In [9]:
if __name__ == '__main__':
    app.run(jupyter_height=920, debug=True, port = 8090)